In [204]:
import pandas as pd
df=pd.read_csv('new_data.csv')

df=df[(df.type_char!='file_folder')&(df.type_char!='handwritten')]

df.type_char.replace(['facture', 'justif_domicile','paye',
       'rrc.cvc', 'form', 'scientific_publication','letter',
       'advertisement', 'resume', 'specification', 'memo',
       'invoice', 'budget', 'news_article', 'presentation',
       'scientific_report', 'email', 'questionnaire'],['texte']*18,inplace=True)

df['type']=df.type_char.replace({'id_pieces':0,'passeport':1,'carte postale':2,'texte':3})
    
df.head()


,Unnamed: 0,filename,type,text_image,type_char
0,0,img_0000000.jpg,3,FACTURE\n\nLOGO\n\nJoanna Binet\n48 Coubertin\...,texte
1,1,img_0000001.jpg,3,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,texte
2,2,img_0000002.jpg,3,FACTURE\n\nMon entreprise : Nom de la société\...,texte
3,3,img_0000003.jpg,3,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,texte
4,4,img_0000004.jpg,3,Payer en ligne >\nFACTURE No\n\nSFIDELI\n\nF/0...,texte


In [205]:
import cv2 as cv
import numpy as np
list_img=[]
for img_n  in df.filename :
    img=cv.imread('final/'+img_n)
    #gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img= cv.resize(img, (64, 64))
    list_img.append(img)
    
X=np.array(list_img)
Y=np.array(df.type)

In [216]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
X_train, X_test, Y_train ,Y_test =train_test_split(X,Y,test_size=0.2,random_state=22)

In [217]:
X_train=list(X_train)
Y_train=list(Y_train)

In [218]:
import albumentations as A
transform = A.Compose([
            A.Rotate(limit=90, p=0.9, border_mode=cv.BORDER_CONSTANT),
            A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=1, p=0.5),
            A.RandomCrop(width=64, height=64),
            A.VerticalFlip(p=0.1),
            A.HorizontalFlip(p=0.5)])


def augment_data(list_x,list_y,nbr_augment):
    
    """augmenter le nombre de données"""
  
    dic_x={}
    X=[]
    Y=[]

    for val_y ,val_x in zip(list_y,list_x):
        X.extend([transform(image=val_x)['image'] for i in range(nbr_augment)])
        X.append(val_x)
        Y.extend([val_y]*(nbr_augment+1))
        
    return np.array(X),np.array(Y)

In [219]:
X_train,Y_train=augment_data(X_train,Y_train,10)
X_test,Y_test=augment_data(X_test,Y_test,2)

In [220]:
X_train=X_train/255
X_test=X_test/255

In [221]:
from keras.utils import to_categorical
#X=X/255
#X_train, X_test, Y_train ,Y_test =train_test_split(X,Y,test_size=0.2)
y_train=to_categorical(Y_train)
y_test=to_categorical(Y_test)

In [222]:
X_train =X_train.reshape((-1,64,64,3))
X_test =X_test.reshape((-1,64,64,3))

In [209]:
%matplotlib inline
import numpy as np

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D


import matplotlib.pyplot as plt

In [223]:
inputs = Input(shape = (64,64,3), name = "Input")
conv_1=Conv2D(filters = 30,kernel_size = (2, 2), padding = 'valid',activation = 'relu')
max_Pool_1=MaxPooling2D(pool_size = (2, 2))
conv_2=Conv2D(filters = 26,kernel_size = (2, 2), padding = 'valid',activation = 'relu')
max_Pool_2=MaxPooling2D(pool_size = (2, 2))
dropout=Dropout(rate = 0.2)
aplat=Flatten()
dense_1=Dense(units =32, activation = "relu")
dense_2=Dense(units = 4, activation = "softmax")
x=conv_1(inputs)
x=max_Pool_1(x)
x=conv_2(x)
x=max_Pool_2(x)
x=dropout(x)
x=aplat(x)
x=dense_1(x)
outputs=dense_2(x)
model= Model(inputs = inputs, outputs = outputs)
model.summary()

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_44 (Conv2D)          (None, 63, 63, 30)        390       
                                                                 
 max_pooling2d_44 (MaxPoolin  (None, 31, 31, 30)       0         
 g2D)                                                            
                                                                 
 conv2d_45 (Conv2D)          (None, 30, 30, 26)        3146      
                                                                 
 max_pooling2d_45 (MaxPoolin  (None, 15, 15, 26)       0         
 g2D)                                                            
                                                                 
 dropout_22 (Dropout)        (None, 15, 15, 26)        0  

In [224]:
model.compile(loss="categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])

training_history_lenet=model.fit(X_train,y_train,epochs =100, batch_size = 200, validation_split = 0.2)


Epoch 1/100
55/55 [==============================] - 21s 368ms/step - loss: 0.8920 - accuracy: 0.6930 - val_loss: 0.7388 - val_accuracy: 0.7457
Epoch 2/100
55/55 [==============================] - 20s 371ms/step - loss: 0.7523 - accuracy: 0.7312 - val_loss: 0.6847 - val_accuracy: 0.7748
Epoch 3/100
55/55 [==============================] - 20s 369ms/step - loss: 0.6968 - accuracy: 0.7461 - val_loss: 0.6756 - val_accuracy: 0.7744
Epoch 4/100
55/55 [==============================] - 20s 361ms/step - loss: 0.6692 - accuracy: 0.7567 - val_loss: 0.6632 - val_accuracy: 0.7821
Epoch 5/100
55/55 [==============================] - 19s 349ms/step - loss: 0.6269 - accuracy: 0.7631 - val_loss: 0.6771 - val_accuracy: 0.7626
Epoch 6/100
55/55 [==============================] - 19s 348ms/step - loss: 0.6051 - accuracy: 0.7684 - val_loss: 0.6559 - val_accuracy: 0.7648
Epoch 7/100
55/55 [==============================] - 19s 351ms/step - loss: 0.6019 - accuracy: 0.7683 - val_loss: 0.6632 - val_accuracy:

55/55 [==============================] - 17s 316ms/step - loss: 0.1567 - accuracy: 0.9465 - val_loss: 1.1458 - val_accuracy: 0.7869
Epoch 58/100
55/55 [==============================] - 17s 316ms/step - loss: 0.1450 - accuracy: 0.9503 - val_loss: 1.3001 - val_accuracy: 0.7733
Epoch 59/100
55/55 [==============================] - 17s 316ms/step - loss: 0.1462 - accuracy: 0.9510 - val_loss: 1.0977 - val_accuracy: 0.7582
Epoch 60/100
55/55 [==============================] - 17s 305ms/step - loss: 0.1532 - accuracy: 0.9484 - val_loss: 1.2401 - val_accuracy: 0.7700
Epoch 61/100
55/55 [==============================] - 17s 317ms/step - loss: 0.1365 - accuracy: 0.9536 - val_loss: 1.2290 - val_accuracy: 0.7777
Epoch 62/100
55/55 [==============================] - 17s 312ms/step - loss: 0.1406 - accuracy: 0.9545 - val_loss: 1.2275 - val_accuracy: 0.7781
Epoch 63/100
55/55 [==============================] - 17s 310ms/step - loss: 0.1438 - accuracy: 0.9519 - val_loss: 1.2372 - val_accuracy: 0.789

In [225]:
from sklearn.metrics import classification_report

test_pred_lenet=model.predict(X_test)
test_pred_lenet_class=test_pred_lenet.argmax(axis=1)
y_test_class=y_test.argmax(axis=1)
print(classification_report(y_test_class, test_pred_lenet_class))

29/29 [==============================] - 1s 16ms/step
              precision    recall  f1-score   support

           0       0.64      0.53      0.58        96
           1       0.41      0.37      0.39        60
           2       0.50      0.50      0.50       123
           3       0.90      0.93      0.91       648

    accuracy                           0.79       927
   macro avg       0.61      0.58      0.60       927
weighted avg       0.79      0.79      0.79       927

